In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp core

# Imports

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *


In [ ]:
#export
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python
import matplotlib
import matplotlib.pyplot as plt # for plotting


# Venn diagram
from matplotlib_venn import venn2
import re
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
eng_stopwords = stopwords.words('english')
import gc

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from fastai2.basics import *
from fastai2.text.all import *
from fastai2.callback.all import *

In [ ]:
#export
import os
print(os.listdir("../data/gquest_data/"))
data_path = Path("../data/gquest_data/")

['test.csv', 'sample_submission.csv', 'train.csv']


In [ ]:
import pdb

# Reading data

In [ ]:
#export
print('Reading data...')
train_data = pd.read_csv(str(data_path/'train/train.csv'))
test_data = pd.read_csv(str(data_path/'test/test.csv'))
sample_submission = pd.read_csv(str(data_path/'sample_submission.csv'))
print('Reading data completed')

Reading data...
Reading data completed


In [ ]:
#export
print('Size of train_data', train_data.shape)
print('Size of test_data', test_data.shape)
print('Size of sample_submission', sample_submission.shape)
test_eq(train_data.shape,(6079,41))
test_eq(test_data.shape,(476,11))
test_eq(sample_submission.shape,(476,31))

Size of train_data (6079, 41)
Size of test_data (476, 11)
Size of sample_submission (476, 31)


In [ ]:
train_data.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes instead of a macro lens?,"After playing around with macro photography on-the-cheap (read: reversed lens, rev. lens mounted on a straight lens, passive extension tubes), I would like to get further with this. The problems with the techniques I used is that focus is manual and aperture control is problematic at best. This limited my setup to still subjects (read: dead insects) Now, as spring is approaching, I want to be able to shoot live insects. I believe that for this, autofocus and settable aperture will be of great help.\n\nSo, one obvious but expensive option is a macro lens (say, EF 100mm Macro) However, I am ...",ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skinny.\n\n\n ...what am I losing when using tubes...?\n\n\nA very considerable amount of light! Increasing that distance from the end of the lens to the sensor can cut your light several stops. Combined with the fact that you'll usually shoot stopped down - expect to need to increase your ISO considerably.\n\nThe fact the macro's are usually considered very very sharp, although I believe that 70-200mm 2.8 is supposed to be quite sharp.\n\nThe ultra low distortion typical of many macros.\n\nI wouldn't worry too much about the bokeh since the DOF...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/what-am-i-losing-when-using-extension-tubes-instead-of-a-macro-lens,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a sprawl/metroplex... between downtown and a commercial district?,"I am trying to understand what kinds of places the spam values on p 231 refer to in the 5th Edition main book for Shadowrun.\n\nPer p 15, a sprawl is a plex, a plex is a ""metropolitan complex, short for metroplex"". Per Google a metroplex is "" a very large metropolitan area, especially one that is an aggregation of two or more cities"". A city downtown and sprawl downtown would tend to have similar densities, but for some reason the sprawl (which includes suburbs?) has a higher spam zone noise rating (p 231). Similarly, I'd think of a downtown as being more dense and noisy (e.g. Office bui...",russellpierce,https://rpg.stackexchange.com/users/8774,"It might be helpful to look into the definition of spam zone:\n\n(p.216) spam zone: An area flooded with invasive and/or viral AR advertising, causing noise.\n\nBecause a metroplex has so many marketing targets, it seems a safe assumption that marketers would drown the plex with spam. Spam from the less dense areas would bleed into the urban cores. A smaller city with less urban/suburban territory surrounding it ostensibly wouldn't have as much spam.\n",Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/what-is-the-distinction-between-a-city-and-a-sprawl-metroplex-between-downtow,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole component pins,"I'm working on a PCB that has through-hole components on both sides of the board. The ""top"" side of the board is mounted flush to a Delrin plastic block (the only top-side component is a gas sensor that is fed air samples through hose fittings in the plastic block).\n\nThe flush mounting means that I have to add grooves to the plastic block to accommodate the soldered pins of the bottom-side components. Assuming a standard 0.062"" thickness FR4 board, how deep do I need to make the grooves in

In [ ]:
train_data.columns

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host', 'question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfa

In [ ]:
sample_submission.columns

Index(['qa_id', 'question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written'],
      dtype='object')

In [ ]:
targets = list(sample_submission.columns[1:])
targets

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written',
 'answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [ ]:
text_columns=['question_title','question_body','answer']

In [ ]:
train_data[targets].describe()

,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,...,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000
mean,0.892663,0.595301,0.057301,0.698525,0.772633,0.793689,0.587478,0.507275,0.238745,0.004469,...,0.799931,0.925408,0.654823,0.960054,0.968626,0.854680,0.479547,0.130641,0.502468,0.908254
std,0.132047,0.219470,0.182196,0.350938,0.303023,0.336622,0.135900,0.185987,0.335057,0.045782,...,0.178420,0.114836,0.107666,0.086926,0.074631,0.130743,0.422921,0.225718,0.407097,0.100708
min,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000,...,0.333333,0.333333,0.333333,0.333333,0.333333,0.200000,0.000000,0.000000,0.000000,0.333333
25%,0.777778,0.444444,0.000000,0.500000,0.666667,0.666667,0.444444,0.333333,0.000000,0.000000,...,0.666667,0.888889,0.666667,1.000000,1.000000,0.800000,0.000000,0.000000,0.000000,0.888889
50%,0.888889,0.555556,0.000000,0.666667,1.000000,1.000000,0.555556,0.444444,0.000000,0.000000,...,0.833333,1.000000,0.666667,1.000000,1.000000,0.866667,0.500000,0.000000,0.500000,0.888889
75%,1.000000,0.777778,0.000000,1.000000,1.000000,1.000000,0.666667,0.666667,0.333333,0.000000,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.933333,1.000000,0.333333,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
train_data['question_body'][0]

'After playing around with macro photography on-the-cheap (read: reversed lens, rev. lens mounted on a straight lens, passive extension tubes), I would like to get further with this. The problems with the techniques I used is that focus is manual and aperture control is problematic at best. This limited my setup to still subjects (read: dead insects) Now, as spring is approaching, I want to be able to shoot live insects. I believe that for this, autofocus and settable aperture will be of great help.\n\nSo, one obvious but expensive option is a macro lens (say, EF 100mm Macro) However, I am not really interested in yet another prime lens. An alternative is the electrical extension tubes.\n\nExcept for maximum focusing distance, what am I losing when using tubes (coupled with a fine lens, say EF70-200/2.8) instead of a macro lens?\n'

In [ ]:
text_columns

['question_title', 'question_body', 'answer']

In [ ]:
df_tokenized,token_count=tokenize_df(train_data,text_columns)

In [ ]:
df_tokenized.head()

,qa_id,question_user_name,question_user_page,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,...,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written,text
0,0,ysap,https://photo.stackexchange.com/users/1024,rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/what-am-i-losing-when-using-extension-tubes-instead-of-a-macro-lens,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,...,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000,"[xxbos, xxfld, 1, xxmaj, what, am, i, losing, when, using, extension, tubes, instead, of, a, macro, lens, ?, xxfld, 2, xxmaj, after, playing, around, with, macro, photography, on, -, the, -, cheap, (, read, :, reversed, lens, ,, rev, ., lens, mounted, on, a, straight, lens, ,, passive, extension, tubes, ), ,, i, would, like, to, get, further, with, this, ., xxmaj, the, problems, with, the, techniques, i, used, is, that, focus, is, manual, and, aperture, control, is, problematic, at, best, ., xxmaj, this, limited, my, setup, to, still, subjects, (, read, :, dead, insects, ), xxmaj, now, ,, ..."
1,1,russellpierce,https://rpg.stackexchange.com/users/8774,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/what-is-the-distinction-between-a-city-and-a-sprawl-metroplex-between-downtow,CULTURE,rpg.stackexchange.com,1.000000,1.000000,...,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889,"[xxbos, xxfld, 1, xxmaj, what, is, the, distinction, between, a, city, and, a, sprawl, /, metroplex, …, between, downtown, and, a, commercial, district, ?, xxfld, 2, i, am, trying, to, understand, what, kinds, of, places, the, spam, values, on, p, 231, refer, to, in, the, 5th, xxmaj, edition, main, book, for, xxmaj, shadowrun, ., \n\n, xxmaj, per, p, 15, ,, a, sprawl, is, a, plex, ,, a, plex, is, a, "", metropolitan, complex, ,, short, for, metroplex, "", ., xxmaj, per, xxmaj, google, a, metroplex, is, "", a, very, large, metropolitan, area, ,, especially, one, that, is, an, aggregation, of, ..."
2,2,Joe Baker,https://electronics.stackexchange.com/users/10157,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions/154225/maximum-protusion-length-for-through-hole-component-pins,SCIENCE,electronics.stackexchange.com,0.888889,0.666667,...,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889,"[xxbos, xxfld, 1, xxmaj, maximum, protusion, length, for, through, -, hole, component, pins, xxfld, 2, xxmaj, i, 'm, working, on, a, xxup, pcb, that, has, through, -, hole, components, on, both, sides, of, the, board, ., xxmaj, the, "", top, "", side, of, the, board, is, mounted, flush, to, a, xxmaj, delrin, plastic, block, (, the, only, top, -, side, component, is, a, gas, sensor, that, is, fed, air, samples, through, hose, fittings, in, the, plastic, block, ), ., \n\n, xxmaj, the, flush, mounting, means, that, i, have, to, add, grooves, to, the, plastic, block, to, accommodate, the, solder..."
3,3,Scimonster,https://judaism.stackexchange.com/users/5151,Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/55182/can-an-affidavit-be-used-in-beit-din,CULTURE,judaism.stackexchange.com,0.888889,0.666667,...,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000,"[xxbos, xxfld, 1, xxmaj, can, an, affidavit, be, used, in, xxmaj, beit, xxmaj, din, ?, xxfld, 2, xxmaj, an, affidavit, ,, from, what, i, understand, ,, is, basically, a, signed, document, given, by, a, witness, to, be, used, as, evidence, in, a, trial, ,, without, the, witness, themselves, needing, to, take, a, stand, ., \n\n, xxmaj, can, an, affidavit, be, used, in, xxmaj, beit, xxmaj, din, ?, xxmaj, or, must,

In [ ]:
vocab = make_vocab(token_count)
vocab

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 'the',
 ',',
 '.',
 'to',
 'a',
 '\n▁',
 ')',
 'i',
 '/',
 '\n\n',
 'is',
 'and',
 'of',
 '"',
 '(',
 'in',
 ':',
 '\n',
 'it',
 '-',
 'that',
 '\\',
 'you',
 '$',
 ';',
 'for',
 '?',
 'this',
 '=',
 '}',
 "'",
 'be',
 'on',
 'with',
 '>',
 '3',
 '1',
 'if',
 'have',
 'as',
 'not',
 '<',
 'can',
 'are',
 '{',
 '2',
 'but',
 'or',
 '\n\n\n',
 'do',
 "'s",
 "n't",
 'my',
 'from',
 ']',
 'an',
 'so',
 'at',
 'your',
 'there',
 'what',
 '\n\n▁',
 'will',
 'would',
 'how',
 'by',
 'one',
 'use',
 'which',
 'when',
 'like',
 'all',
 '0',
 'some',
 '&',
 'then',
 '[',
 'they',
 'any',
 'get',
 'does',
 'using',
 'was',
 '*',
 'just',
 'more',
 '+',
 'has',
 'should',
 'we',
 'want',
 '#',
 'other',
 "'m",
 'need',
 'only',
 'no',
 'am',
 'way',
 'about',
 'also',
 'up',
 'new',
 'time',
 'could',
 'out',
 'code',
 '…',
 'me',
 '!',
 'where',
 'here',
 'them',
 'same',
 'make',
 'than',
 'file',
 'know',

In [ ]:
#export
def testeFunc(message):
    "doc string test"
    return(f'The test message is: {message}')

In [ ]:
test_eq(testeFunc("ok!"),"The test message is: ok!")

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
